![train](http://3.bp.blogspot.com/-Nt8l3SY2UiY/VHHLVsm8cFI/AAAAAAAAJGg/sPXvFN41aaU/s1600/contr%C3%A1rio.jpg)

In [1]:
import numpy as np
np.random.seed(1019)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import xgboost

import sklearn
from sklearn.model_selection import train_test_split 

import sys, os, gc, types
import time
from subprocess import check_output

In [2]:
sys.path.append('./utils')

from training import cv, train
from plotting import plot_importance
from data import Data

In [3]:
root_paths = [
    "/data/kaggle-instacart",
    "/Users/jiayou/Dropbox/珺珺的程序/Kaggle/Instacart",
    "/Users/jiayou/Dropbox/Documents/珺珺的程序/Kaggle/Instacart"
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break

In [4]:
def predict(bst):
    d_test = xgboost.DMatrix(
        test.drop(['eval_set', 'order_id', 'reordered', 'product_id'], axis=1))
    return bst.predict(d_test)

def ensemble(preds):
    # Average ensemble
    r = None
    for p in preds:
        if r is None:
            r = p
        else:
            r += p
    return r / len(preds)

def ensemble_predict(bsts):
    preds = []
    for bst in bsts:
        preds.append(predict(bst))
    return ensemble(preds)

def thresholding(pred):
    return (pred > 0.21).astype(int)

In [5]:
name = 'v3-test'
down_sample = 10
num_searches = 1

In [6]:
bsts = []
for i in range(num_searches):
    bsts.append(xgboost.Booster(model_file=os.path.join(root, 'train-{}-n{}.bst'.format(name, i))))

In [7]:
test = Data.test(down_sample=down_sample)

In [8]:
# Prediction
test['reordered'] = thresholding(ensemble_predict([bst for bst in bsts]))

In [9]:
# Submission file
test['product_id'] = test.product_id.astype(str)
submit = test[test.reordered == 1].groupby('order_id')['product_id'].agg([lambda x: ' '.join(set(x))]).reset_index()
sample_submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))
submit.columns = sample_submission.columns.tolist()
submit_final = sample_submission[['order_id']].merge(submit, how='left').fillna('None')
submit_final.to_csv("submission-{}.csv".format(name), index=False)

In [10]:
# Stats
print('{} pred orders; {} of them non-empty'.format(submit_final.shape[0], submit.shape[0]))
empty_order_ratio = (submit_final.shape[0] - submit.shape[0]) * 100. / submit_final.shape[0]
print('Empty order ratio is {:.2f}%'.format(empty_order_ratio))

75000 pred orders; 743 of them non-empty
Empty order ratio is 99.01%
